In [ ]:
extract = []
file = open("data.txt", "r")
for i in file:
    extract.append(i)
    

In [ ]:
print(f"Size of list before removal of duplicates: {len(extract)}")
res = [*set(extract)]
print(f"Size of list after removal of duplicates: {len(res)}")
print(f"Number of duplicates removed: {len(extract) - len(res)}")

-1 => negative <br>
0 => neutral <br>
1 => positive <br>

In [ ]:
new = []
for i in res:
    if "negative" in i:
        new.append(i.replace("negative", "-1"))
    elif "positive" in i:
        new.append(i.replace("positive", "1"))
    elif "neutral" in i:
        new.append(i.replace("neutral", "0"))

In [ ]:
file = open("cleaned_data.txt", "w+")
for i in new:
    file.write(i)
file.close()

In [ ]:
!rm data.txt
!mv cleaned_data.txt data.txt

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def remove_stop_words(text):
    words = text.split()
    words_without_stop_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(words_without_stop_words)

with open('data.txt', 'r') as file:
    lines = file.readlines()
    preprocessed_data = []
    for line in lines:
        values = line.strip().split(',')
        if len(values) != 2:
            print(f'Error: Unexpected number of values ({len(values)}) in line: {line}')
            continue
        label, text = values
        text = text.strip().strip('"')
        preprocessed_text = remove_stop_words(text)
        preprocessed_data.append((label, preprocessed_text))

with open('preprocessed_data.txt', 'w') as file:
    for label, text in preprocessed_data:
        file.write(f'{label},"{text}"\n')


In [ ]:
import re

def remove_punctuation_and_special_characters(text):
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

preprocessed_data = []
with open("preprocessed_data.txt") as f:
    for line in f:
        label, text = line.strip().split(',', 1)
        text = remove_punctuation_and_special_characters(text)
        preprocessed_data.append((label, text))

# Save the cleaned text data to a file
with open("result.txt", "w") as f:
    for label, text in preprocessed_data:
        f.write("{},{}\n".format(label, text))


In [ ]:
preprocessed_data = []
with open("result.txt") as f:
    for line in f:
        try:
            label, text = line.strip().split(',', 1)
            if label == "positive":
                label = "1"
            label = int(label.strip('"'))
            preprocessed_data.append([label, text])
        except:
            print(f"Error: {line}")
#        print(f"data appeneded: {label} {text}")

print(preprocessed_data)

with open("result_data.txt", "w") as f:
    for label, text in preprocessed_data:
        print(label, text)
        f.write(f"{label},{text}\n")


In [10]:
#tokenize the data

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

preprocessed_data = []
with open("result_data.txt") as f:
    for line in f:
        label, text = line.strip().split(',', 1)
        label = int(label)
        text = word_tokenize(text)
        preprocessed_data.append((label, text))

with open("tokenized_data.txt", "w") as f:
    for label, text in preprocessed_data:
        f.write(f"{label}:{' '.join(text)}\n")


[nltk_data] Downloading package punkt to /home/nav/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<h3>converting text to numerical data<br></h3>

In [18]:
from gensim.models import Word2Vec
from tqdm import tqdm

def run_word2vec(tokenized_data_file, size=100, window=5, min_count=5, workers=4):
    # load the tokenized data
    with open(tokenized_data_file, "r") as f:
        corpus = [line.strip().split() for line in f.readlines()]
        
    # train the word2vec model
    model = Word2Vec(corpus, vector_size=size, window=window, min_count=min_count, workers=workers, sg=1)
    
    # save the model
    model.save("word2vec.model")
    
    # return the trained model
    return model

# call the run_word2vec function
model = run_word2vec("tokenized_data.txt")


In [3]:
import numpy as np
from tqdm import tqdm
from gensim.models import Word2Vec

word2vec_model = Word2Vec.load("word2vec.model")

def sentence_to_vec(sentence):
    words = sentence.split()
    feature_vec = np.zeros((word2vec_model.vector_size,), dtype="float32")
    n_words = 0
    for word in words:
        if word in word2vec_model.wv:
            n_words = n_words + 1
            feature_vec = np.add(feature_vec, word2vec_model.wv[word])
    if n_words > 0:
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

def save_feature_vectors(word2vec_model, tokenized_data_file, feature_vector_file):
    with open(tokenized_data_file, 'r') as file:
        lines = file.readlines()
        
    with open(feature_vector_file, 'w') as file:
        for line in tqdm(lines):
            label, text = line.strip().split(':')
            words = text.split()
            feature_vector = np.zeros((word2vec_model.vector_size,), dtype="float32")
            n_words = 0
            for word in words:
                if word in word2vec_model.wv:
                    n_words = n_words + 1
                    feature_vector = np.add(feature_vector, word2vec_model.wv[word])
            if n_words > 0:
                feature_vector = np.divide(feature_vector, n_words)
            feature_vector = ' '.join(str(x) for x in feature_vector)
            file.write(f'{label} {feature_vector}\n')

save_feature_vectors(word2vec_model, 'tokenized_data.txt', 'feature_vectors.txt')


100%|██████████| 1255135/1255135 [01:04<00:00, 19446.80it/s]


<h3>Separating test and training data</h3>

In [4]:
import random

def separate_data(file_path, ratio=0.8):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    random.shuffle(lines)
    num_training = int(ratio * len(lines))
    training_data = lines[:num_training]
    testing_data = lines[num_training:]
    return training_data, testing_data

training_data, testing_data = separate_data('feature_vectors.txt')

with open('training_data.txt', 'w') as f:
    f.writelines(training_data)

with open('testing_data.txt', 'w') as f:
    f.writelines(testing_data)
